In [ ]:
!pip install google-cloud-storage
!pip install folium ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.8 MB/s eta 0:00:00


In [25]:
from google.cloud import storage
from google.colab import files

In [ ]:
 storage_client = storage.Client()

In [24]:
# Manipulación de datos
import numpy as np
import pandas as pd
from io import BytesIO
import os
import folium
from folium.plugins import MarkerCluster
from ipywidgets import interact, widgets
from IPython.display import display, clear_output

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento y machine learning (scikit-learn)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error, r2_score

# Modelos supervisados más comunes
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBRegressor

# Configuración de visualizaciones
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

In [13]:
archivos_2025 = ['01-2025.csv', '02-2025.csv']

dataframes = []
for archivo in archivos_2025:
    df = pd.read_csv(archivo, sep=';')  # separador ;
    dataframes.append(df)


df_2025 = pd.concat(dataframes, ignore_index=True)

print(df_2025.head())

     id                fecha tipo_elem  intensidad  ocupacion  carga  vmed  \
0  1002  2025-01-01 00:00:00       C30         216          2      0  66.0   
1  1002  2025-01-01 00:15:00       C30          72          0      0  14.0   
2  1002  2025-01-01 00:30:00       C30         444          2      0  69.0   
3  1002  2025-01-01 00:45:00       C30        1260          6      0  66.0   
4  1002  2025-01-01 01:00:00       C30        2028          7      0  66.0   

  error  periodo_integracion  
0     N                    5  
1     N                    5  
2     N                    5  
3     N                    5  
4     N                    5  


In [16]:
# prompt: buscar los valores unicos de la columna id

unique_ids = df_2025['id'].unique()
print(unique_ids)
count_unique_ids = len(unique_ids)
print(count_unique_ids)


[ 1002  1003  1006 ... 10962 11105 11394]
4725


In [27]:
ubicacion_2025 = pd.read_csv('ubicacion_2025.csv', sep=';', encoding='latin-1') # Changed encoding to 'latin-1'

print(ubicacion_2025.head())



  tipo_elem  distrito    id cod_cent   nombre          utm_x         utm_y  \
0       M30       8.0  6639  PM40001  PM40001  443088.625859  4.482547e+06   
1       M30       8.0  3797  PM40005  PM40005  443081.764312  4.482550e+06   
2       M30       5.0  6640  PM10013  PM10013  442865.853275  4.481386e+06   
3       M30       5.0  6642  PM10091  PM10091  442835.412762  4.480813e+06   
4       M30       5.0  6643  PM10092  PM10092  442819.699444  4.480807e+06   

   longitud    latitud  
0 -3.671563  40.491676  
1 -3.671644  40.491700  
2 -3.674087  40.481200  
3 -3.674395  40.476038  
4 -3.674579  40.475980  


In [30]:
# prompt: añdir una columna a ubicacion_2025 con la zona correspondiene:Diccionario con coordenadas de zonas
coordenadas_zonas = { # Define the coordenadas_zonas dictionary
    "Zona 01 (Interior M-30)": (40.4168, -3.7038),
    "Zona 02 (Sureste)": (40.3700, -3.5800),
    "Zona 03 (Noreste)": (40.5100, -3.6300),
    "Zona 04 (Noroeste)": (40.4500, -3.8000),
    "Zona 05 (Suroeste)": (40.3300, -3.8200)
}
# según sus columnas de 'longitud' y 'latitud'

def asignar_zona(fila, coordenadas_zonas):
    """Asigna una zona a una fila del DataFrame basándose en su latitud y longitud."""
    latitud = fila['latitud']
    longitud = fila['longitud']

    min_distancia = float('inf')
    zona_asignada = None

    for zona, (lat_zona, lon_zona) in coordenadas_zonas.items():
        distancia = np.sqrt((latitud - lat_zona)**2 + (longitud - lon_zona)**2)
        if distancia < min_distancia:
            min_distancia = distancia
            zona_asignada = zona

    return zona_asignada

# Aplicar la función a cada fila del DataFrame
ubicacion_2025['zona'] = ubicacion_2025.apply(asignar_zona, args=(coordenadas_zonas,), axis=1)

print(ubicacion_2025.head())

  tipo_elem  distrito    id cod_cent   nombre          utm_x         utm_y  \
0       M30       8.0  6639  PM40001  PM40001  443088.625859  4.482547e+06   
1       M30       8.0  3797  PM40005  PM40005  443081.764312  4.482550e+06   
2       M30       5.0  6640  PM10013  PM10013  442865.853275  4.481386e+06   
3       M30       5.0  6642  PM10091  PM10091  442835.412762  4.480813e+06   
4       M30       5.0  6643  PM10092  PM10092  442819.699444  4.480807e+06   

   longitud    latitud               zona  
0 -3.671563  40.491676  Zona 03 (Noreste)  
1 -3.671644  40.491700  Zona 03 (Noreste)  
2 -3.674087  40.481200  Zona 03 (Noreste)  
3 -3.674395  40.476038  Zona 03 (Noreste)  
4 -3.674579  40.475980  Zona 03 (Noreste)  


In [36]:
#crear un nuevo df a partir de ubicacion_2025	con 'tip_elem', 'distrito', 'id' y 'zona'

df_ubicacion_2025 = ubicacion_2025[['tipo_elem', 'distrito', 'id', 'zona']]

print(df_ubicacion_2025.head(50))

   tipo_elem  distrito     id                     zona
0        M30       8.0   6639        Zona 03 (Noreste)
1        M30       8.0   3797        Zona 03 (Noreste)
2        M30       5.0   6640        Zona 03 (Noreste)
3        M30       5.0   6642        Zona 03 (Noreste)
4        M30       5.0   6643        Zona 03 (Noreste)
5        M30       5.0   6644        Zona 03 (Noreste)
6        M30       5.0   6645        Zona 03 (Noreste)
7        M30       5.0   6646        Zona 03 (Noreste)
8        M30       5.0   6647        Zona 03 (Noreste)
9        M30       5.0   6648        Zona 03 (Noreste)
10       M30       5.0   6649  Zona 01 (Interior M-30)
11       M30       5.0   6650  Zona 01 (Interior M-30)
12       M30       5.0   6651  Zona 01 (Interior M-30)
13       M30       5.0   6652  Zona 01 (Interior M-30)
14       M30       5.0   6653  Zona 01 (Interior M-30)
15       M30       5.0   6654  Zona 01 (Interior M-30)
16       M30       5.0   6655  Zona 01 (Interior M-30)
17       M

In [37]:
# prompt: descargar en un .csv df_ubicacion_2025

df_ubicacion_2025.to_csv('df_ubicacion_2025.csv', index=False)
from google.colab import files
files.download('df_ubicacion_2025.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
# prompt: añadir columna 'zona' del df_ubicación_2025 al df_2025 concatenando con la id

# Merge the dataframes based on the 'id' column
df_2025 = pd.merge(df_2025, df_ubicacion_2025, on='id', how='left')

print(df_2025.head())


     id                fecha tipo_elem_x  intensidad  ocupacion  carga  vmed  \
0  1002  2025-01-01 00:00:00         C30         216          2      0  66.0   
1  1002  2025-01-01 00:15:00         C30          72          0      0  14.0   
2  1002  2025-01-01 00:30:00         C30         444          2      0  69.0   
3  1002  2025-01-01 00:45:00         C30        1260          6      0  66.0   
4  1002  2025-01-01 01:00:00         C30        2028          7      0  66.0   

  error  periodo_integracion tipo_elem_y  distrito                     zona  
0     N                    5       other      10.0  Zona 01 (Interior M-30)  
1     N                    5       other      10.0  Zona 01 (Interior M-30)  
2     N                    5       other      10.0  Zona 01 (Interior M-30)  
3     N                    5       other      10.0  Zona 01 (Interior M-30)  
4     N                    5       other      10.0  Zona 01 (Interior M-30)  


In [39]:
# prompt: eliminar columnas 'error',  'periodo_integracion',  'tipo_elem_y' y  'distrito'

# Eliminar columnas especificadas
df_2025 = df_2025.drop(columns=['error', 'periodo_integracion', 'tipo_elem_y', 'distrito'], errors='ignore')

print(df_2025.head())


     id                fecha tipo_elem_x  intensidad  ocupacion  carga  vmed  \
0  1002  2025-01-01 00:00:00         C30         216          2      0  66.0   
1  1002  2025-01-01 00:15:00         C30          72          0      0  14.0   
2  1002  2025-01-01 00:30:00         C30         444          2      0  69.0   
3  1002  2025-01-01 00:45:00         C30        1260          6      0  66.0   
4  1002  2025-01-01 01:00:00         C30        2028          7      0  66.0   

                      zona  
0  Zona 01 (Interior M-30)  
1  Zona 01 (Interior M-30)  
2  Zona 01 (Interior M-30)  
3  Zona 01 (Interior M-30)  
4  Zona 01 (Interior M-30)  


In [42]:
# prompt: eliminamos las horas de la columna fechas

# Convert 'fecha' column to datetime objects
df_2025['fecha'] = pd.to_datetime(df_2025['fecha'])

# Extract only the date part
df_2025['fecha'] = df_2025['fecha'].dt.date

print(df_2025.head())


     id       fecha tipo_elem_x  intensidad  ocupacion  carga  vmed  \
0  1002  2025-01-01         C30         216          2      0  66.0   
1  1002  2025-01-01         C30          72          0      0  14.0   
2  1002  2025-01-01         C30         444          2      0  69.0   
3  1002  2025-01-01         C30        1260          6      0  66.0   
4  1002  2025-01-01         C30        2028          7      0  66.0   

                      zona  
0  Zona 01 (Interior M-30)  
1  Zona 01 (Interior M-30)  
2  Zona 01 (Interior M-30)  
3  Zona 01 (Interior M-30)  
4  Zona 01 (Interior M-30)  


In [61]:
# prompt: eliminar 'id' y 'tipo_elem_x

# Eliminar columnas especificadas
df_2025 = df_2025.drop(columns=['id', 'tipo_elem_x'], errors='ignore')

print(df_2025.head())


        fecha  intensidad  ocupacion  carga  vmed                     zona
0  2025-01-01         216          2      0  66.0  Zona 01 (Interior M-30)
1  2025-01-01          72          0      0  14.0  Zona 01 (Interior M-30)
2  2025-01-01         444          2      0  69.0  Zona 01 (Interior M-30)
3  2025-01-01        1260          6      0  66.0  Zona 01 (Interior M-30)
4  2025-01-01        2028          7      0  66.0  Zona 01 (Interior M-30)


In [63]:
# prompt: crear un nuevo df calculando la intensidad media con todos los datos del
# mismo dia y zona (dejando así solo una fila por cada fecha y zona), añadiendo columna
# intensidad max y intesidad min y lo mismo con ocupación y vmed. Y tendremos un nuevo
# df con zona, fecha y las columnas calculadas. Así reducimos el tamaño del dataset

# Group data by 'id', 'zona', and 'fecha' and calculate aggregate statistics
df_grouped = df_2025.groupby(['zona', 'fecha']).agg(
    intensidad_media=('intensidad', 'mean'),
    intensidad_max=('intensidad', 'max'),
    intensidad_min=('intensidad', 'min'),
    ocupacion_media=('ocupacion', 'mean'),
    ocupacion_max=('ocupacion', 'max'),
    ocupacion_min=('ocupacion', 'min'),
    vmed_media=('vmed', 'mean'),
    vmed_max=('vmed', 'max'),
    vmed_min=('vmed', 'min')
).reset_index()

# Rename the 'tipo_elem_x' column to 'tipo_elem'
df_grouped = df_grouped.rename(columns={'tipo_elem_x': 'tipo_elem'})

print(df_grouped.head())



                      zona       fecha  intensidad_media  intensidad_max  \
0  Zona 01 (Interior M-30)  2025-01-01        253.453132            6564   
1  Zona 01 (Interior M-30)  2025-01-02        354.704248            7336   
2  Zona 01 (Interior M-30)  2025-01-03        370.334334            7793   
3  Zona 01 (Interior M-30)  2025-01-04        311.803418            6560   
4  Zona 01 (Interior M-30)  2025-01-05        272.535390            5927   

   intensidad_min  ocupacion_media  ocupacion_max  ocupacion_min  vmed_media  \
0               0         3.220079            100              0    6.064263   
1               0         5.013018            100              0    5.773685   
2               0         5.277064            100              0    5.831184   
3               0         4.302878            100              0    5.971468   
4               0         3.639043            100              0    6.044444   

   vmed_max  vmed_min  
0     117.0       0.0  
1     109.0   

In [66]:
df_grouped.groupby(['zona', 'fecha',]).ngroups
len(df_grouped)


295

In [67]:
print(df_grouped.shape)

(295, 11)


In [68]:

print(df_grouped.columns)
print(df_grouped.dtypes)
print(df_grouped.describe())

Index(['zona', 'fecha', 'intensidad_media', 'intensidad_max', 'intensidad_min',
       'ocupacion_media', 'ocupacion_max', 'ocupacion_min', 'vmed_media',
       'vmed_max', 'vmed_min'],
      dtype='object')
zona                 object
fecha                object
intensidad_media    float64
intensidad_max        int64
intensidad_min        int64
ocupacion_media     float64
ocupacion_max         int64
ocupacion_min         int64
vmed_media          float64
vmed_max            float64
vmed_min            float64
dtype: object
       intensidad_media  intensidad_max  intensidad_min  ocupacion_media  \
count        295.000000      295.000000           295.0       295.000000   
mean         313.374237     7109.661017             0.0         4.738436   
std           68.646748     6823.628197             0.0         1.372029   
min          170.512135     1920.000000             0.0         1.851902   
25%          274.350247     3216.500000             0.0         3.792613   
50%          3

In [69]:
print(df_grouped.head(200))

                        zona       fecha  intensidad_media  intensidad_max  \
0    Zona 01 (Interior M-30)  2025-01-01        253.453132            6564   
1    Zona 01 (Interior M-30)  2025-01-02        354.704248            7336   
2    Zona 01 (Interior M-30)  2025-01-03        370.334334            7793   
3    Zona 01 (Interior M-30)  2025-01-04        311.803418            6560   
4    Zona 01 (Interior M-30)  2025-01-05        272.535390            5927   
..                       ...         ...               ...             ...   
195       Zona 04 (Noroeste)  2025-01-19        273.412981            5252   
196       Zona 04 (Noroeste)  2025-01-20        383.706430            5573   
197       Zona 04 (Noroeste)  2025-01-21        388.766866            5901   
198       Zona 04 (Noroeste)  2025-01-22        397.197571            6684   
199       Zona 04 (Noroeste)  2025-01-23        409.636034            6045   

     intensidad_min  ocupacion_media  ocupacion_max  ocupacion_

In [70]:
# prompt: Descargar df_grouped como trafico_zonas_2025.csv

df_grouped.to_csv('trafico_zonas_2025.csv', index=False)
files.download('trafico_zonas_2025.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>